In [1]:
import librosa
import numpy as np
import pandas as pd
from os import listdir
from os.path import isfile, join

# Rutas de las carpetas que contienen los archivos de audio
audio_folders = ["./Output/Adelante", "./Output/Atrás", "./Output/Izquierda", "./Output/Derecha"]

# Lista para almacenar los datos
data = []

# Recorrer cada carpeta de audio
for folder_path in audio_folders:
    audio_files = [f for f in listdir(folder_path) if isfile(join(folder_path, f))]
    
    # Obtener la etiqueta basada en el nombre de la carpeta
    label = folder_path.split("/")[-1]
    
    # Recorrer cada archivo de audio en la carpeta
    for audio_file in audio_files:
        file_path = join(folder_path, audio_file)

        # Cargar el archivo de audio usando librosa
        y, sr = librosa.load(file_path)

        # Calcular las características de audio utilizando librosa
        chroma_stft_mean = np.mean(librosa.feature.chroma_stft(y=y, sr=sr))
        chroma_stft_var = np.var(librosa.feature.chroma_stft(y=y, sr=sr))
        rms_mean = np.mean(librosa.feature.rms(y=y))
        rms_var = np.var(librosa.feature.rms(y=y))
        spectral_centroid_mean = np.mean(librosa.feature.spectral_centroid(y=y, sr=sr))
        spectral_centroid_var = np.var(librosa.feature.spectral_centroid(y=y, sr=sr))
        spectral_bandwidth_mean = np.mean(librosa.feature.spectral_bandwidth(y=y, sr=sr))
        spectral_bandwidth_var = np.var(librosa.feature.spectral_bandwidth(y=y, sr=sr))
        # Agregar más características aquí...

        # Calcular las nuevas características
        rolloff_mean = np.mean(librosa.feature.spectral_rolloff(y=y, sr=sr))
        rolloff_var = np.var(librosa.feature.spectral_rolloff(y=y, sr=sr))
        zero_crossing_rate_mean = np.mean(librosa.feature.zero_crossing_rate(y))
        zero_crossing_rate_var = np.var(librosa.feature.zero_crossing_rate(y))
        harmony_mean = np.mean(librosa.effects.harmonic(y))
        harmony_var = np.var(librosa.effects.harmonic(y))
        perceptr_mean = np.mean(librosa.effects.percussive(y))
        perceptr_var = np.var(librosa.effects.percussive(y))
        tempo, _ = librosa.beat.beat_track(y=y, sr=sr)

        # Calcular los coeficientes MFCC
        mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=20)
        mfcc_means = np.mean(mfccs, axis=1)
        mfcc_vars = np.var(mfccs, axis=1)

        # Agregar los valores a la lista de datos
        data.append([chroma_stft_mean, chroma_stft_var, rms_mean, rms_var,
                     spectral_centroid_mean, spectral_centroid_var, spectral_bandwidth_mean, spectral_bandwidth_var,
                     rolloff_mean, rolloff_var, zero_crossing_rate_mean, zero_crossing_rate_var,
                     harmony_mean, harmony_var, perceptr_mean, perceptr_var,
                     tempo] + list(mfcc_means) + list(mfcc_vars) + [label])

# Crear un DataFrame de pandas con los datos
columns = ['chroma_stft_mean', 'chroma_stft_var', 'rms_mean', 'rms_var',
           'spectral_centroid_mean', 'spectral_centroid_var', 'spectral_bandwidth_mean', 'spectral_bandwidth_var',
           'rolloff_mean', 'rolloff_var', 'zero_crossing_rate_mean', 'zero_crossing_rate_var',
           'harmony_mean', 'harmony_var', 'perceptr_mean', 'perceptr_var',
           'tempo'] + [f'mfcc{i}_mean' for i in range(1, 21)] + [f'mfcc{i}_var' for i in range(1, 21)] + ['label']
df = pd.DataFrame(data, columns=columns)

# Guardar el DataFrame en un archivo CSV
output_csv = "features_with_labels.csv"
df.to_csv(output_csv, index=False)

print("Características extraídas y etiquetas guardadas en: ", output_csv)


Características extraídas y etiquetas guardadas en:  features_with_labels.csv
